In [1]:
# Skriptas rpeaks sekų, gautų iš .json failo ir EKG įrašo, panaudojant NEUROKIT2, sulyginimui
# Nuskaitomi įrašų failai .npy 

from xmlrpc.client import SYSTEM_ERROR
import pandas as pd
import numpy as np
import json, shutil, sys
import neurokit2 as nk
from pathlib import Path

def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.5*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

def AnalyseHeartrate(ecg_signal_df):
    _, rpeaks = nk.ecg_peaks(ecg_signal_df['orig'], sampling_rate=200, method="neurokit", correct_artifacts=False)
    ret = {'rpeaks':rpeaks['ECG_R_Peaks'].tolist()}
    return ret 

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')
rec_dir_eval = Path(db_path, 'records_evaluated')

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
# file_path = Path(rec_dir,'info_create_z.json')
# with open(file_path) as json_file:
#     info_create = json.load(json_file)
# SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas


print("Skriptas rpeaks sulyginimui. Lyginamas rpeaks masyvas, suformuotas iš json failo,")
print("nuskaityto iš Zive frontendo ir rpeaks masyvas, gautas iš EKG signalo naudojant")
print("Neurokit2 funkciją ecg_peaks. EKG signalas (įrašas) irgi gautas iš Zive frontendo.")
print("Abu rpeaks masyvai įrašomi į json vizualiniam sulyginimui su http://www.jsondiff.com/")
print("rpeaks iš json failo varde pridėta '-js', rpeaks iš signalo - pridėta '-sg'")

# Įrašai, kur rpeak nesutapimas tik 1
list = ["1636423.936"]
list = ['1636490.526']
list = ['1636495.646']
list = ['1636499.394']
list = ['1632133.116']
list = ['1636563.972']
list = ['1636593.000']
list = ['1631141.137']
list = ['1631129.850']
list = ['1631082.735']
list = ['1631082.118']
list = ['1631081.543']
list = ['1637166.430']
list = ['1637178.685']
list = ['1633540.147']
list = ['1633958.174']
list = ['1630717.159']
list = ['1632123.567']
list = ['1632728.750'] # 10 Rpeaks, kurių Neurokit nesukuria, mediko sudėta, visi yra V, ML irgi pagauna
list = ['1643061.340'] # 10 rpeaks, ten, kur jų nėra, mediko panaikinta

list = ['1636451.235'] # Važinėjanti izolinija
list = ['1631066.094'] # Yra triukšmelis ties 9800. Nuėmiau rpeaks ties 49000 ir 50800, neaiškumas ties 75400
list = ['1637584.237'] # Pavirpėjimas prie 109200
list = ['1631047.940'] # 
list = ['1631039.807'] # 
list = ['1637584.866'] # 
list = ['1637585.491'] # 
list = ['1631027.284'] # 
list = ['1637619.150'] # Labai važinėjanti izolinija, kartais staigiai, raumenų triukšmų palyginus nedaug 
list = ['1631002.473'] # 

list = ['1633444.221'] # Yra izolinijos staigių pavaikščiojimų, raumenų triukšmų išlindimų. Dažnai N arba S ML atpažista, kaip V.
list = ['1630999.344'] # 0, Yra tik vienoje vietoje staigus izolinijos šoktelėjimas su triukšmų išlindimu ties 119000, ML V gaudo neblogai, yra dubletų, tripletų
list = ['1637628.331'] # 0, Nemažai S, ML juos neblogai gaudo
list = ['1637624.073'] # 3, Reikia iš naujo sudėti rpeaks !!!!
list = ['1637628.402'] # 0, Iššoka triukšmai tik ties 48000.
list = ['1637631.801'] # 0
list = ['1637189.804'] # 0, Reiktų parodyti gydytojui. išmestas rpeak 72858, nors man atrodo, kad jis ten yra, ir ten ekstrasistolė. Yra ir daugiau vietų, atrodo, kaip ekstrasistolės, tačiau nepažymėta. Reiktų parodyti gydytojui.
list = ['1636414.518'] # 0, Sutriukšmauja tik ties 81200 ir 119200
list = ['1634736.586'] # 1, Nestabili izolinija, kartais išlenda triukšmai
list = ['1632923.661'] # 1, Labai ilgas, > 4000. Praleistas pikas, gydytojas pažymėjo. Kokybė nėra aiški
list = ['1630844.567'] # 2

# 2023 01 13 trūko 1:30
list = ['1633443.596'] # 1, Gera kokybė suprastėja nuo 93000, daug S, ML nemažai S ir N priskiria V
list = ['1633434.823'] # 1, Buvo praleistas rpeaks, gydytojas įdėjo. Yra staigių izolinijos šokinėjimų.
list = ['1633434.198'] # 0
list = ['1633427.309'] # 2, Ties 30400 prasideda izolinijos  šokinėjimas
list = ['1633548.426'] # 0, Yra pašokinėjimų nuo 30000, bet nedaug
list = ['1633408.989'] # 0
list = ['1633408.480'] # 2, Yra gana didelės amplitudės raumenų triukšmų
list = ['1633396.439'] # 2
list = ['1634648.967'] # 0
list = ['1633395.185'] # 1
list = ['1633387.028'] # 1

# list = ['1637624.073'] # nesutampančių 1481, bus taisomas specialiomis priemonėmis


# Ar perrašome vertinamą failą į vertinamų failų aplanką?

flag_eval_copy = False 

for file_name in list:
    # file_name = str(file_name)    
    print(f"\nZive įrašas:  {file_name:>2}")

    if (flag_eval_copy):
        # Perrašome failus į vertinamų failų aplanką
        src = Path(rec_dir, file_name)
        dst = Path(rec_dir_eval, file_name)

        # kopijuojame duomenis ir .json failus, išsaugodami metaduomenis su copy2
        shutil.copy2(src, dst)
        src = Path(rec_dir, file_name + '.json')
        dst = Path(rec_dir_eval, file_name + '.json')
        shutil.copy2(src, dst)
        print(f"\nFailai {file_name} ir {file_name+ '.json'} perrašyti  iš {rec_dir} į {rec_dir_eval}")
        print()

        # I-a dalis: nuskaitome rpeaks iš json failo

    filepath = Path(rec_dir, file_name + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_dict = data['rpeaks']
    rpeaks_from_json = np.array([dict['sampleIndex'] for dict in rpeaks_dict])
    # rpeaks_from_json = np.array([3,5,7,10,11,12,13,15,17,19,20,22,23,24])
    # print(rpeaks_from_json)

    print(f"rpeaks iš json failo: {len(rpeaks_from_json)}")
    # print(rpeaks_json) 

    # Suformuojame supaprastintą json failą sulyginimui, prie vardo pridėta 'js'
    # Sulyginimui naudoju http://www.jsondiff.com/
    # data = [{"sampleIndex":int(c), "annotationValue":"N"} for c in rpeaks_from_json]
    # filename = file_name + '-js' + '.json'
    # with open(filename, 'w', encoding='UTF-8') as outfile:
    #     json.dump(data, outfile)
    # print(data)    


        # II-a dalis: suformuojame rpeaks su Neurokitu,
        # pakartojant Zive skriptą iš analysis.py ir heartrate_analysis.py
    filepath = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(filepath)
    # signal = nk.signal_filter(signal=signal_raw, sampling_rate=200, lowcut=0.5, method="butterworth", order=5)
    signal = signal_raw    
    ecg_signal_df = pd.DataFrame(signal, columns=['orig'])

    analysis_results = AnalyseHeartrate(ecg_signal_df)
    rpeaks_from_signal = analysis_results['rpeaks']
    print(f"rpeaks iš signal: {len(rpeaks_from_signal)}")

    # print(rpeaks_from_signal)
    
  # Suformuojame supaprastintą json failą sulyginimui, prie vardo pridėta 'sg'
    # data = [{"sampleIndex":int(c), "annotationValue":"N"} for c in rpeaks_from_signal]
    # filename = file_name + '-sg' + '.json'
    # with open(filename, 'w', encoding='UTF-8') as outfile:
    #     json.dump(data, outfile)
    # print(data)    


            # Sulyginimas

    print("\nReikšmės faile rpeaks_from_json kurių nėra faile rpeaks_from_signal")
    ab = np.setdiff1d(rpeaks_from_json, rpeaks_from_signal)
    # https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html
    # print(ab)   # Return the unique values in a that are not in b

    flag = np.size(ab)
    if flag:
        sorter = np.argsort(rpeaks_from_json)
        idxs = sorter[np.searchsorted(rpeaks_from_json, ab, sorter=sorter)]
        print(f"i     index   rpeak")
        for i in range(len(idxs)):
            print(f"{i} {idxs[i]:>9} {ab[i]:>7}")
        # print(idxs)
    else:
        print('Tokių reikšmių nėra')

    print("Reikšmės faile rpeaks_from_signal kurių nėra faile rpeaks_from_json")
    ba = np.setdiff1d(rpeaks_from_signal,rpeaks_from_json)
    # print(ba)   # Return the unique values in b that are not in a

    flag = np.size(ba)
    if flag:
        sorter = np.argsort(rpeaks_from_signal)
        idxs = sorter[np.searchsorted(rpeaks_from_signal, ba, sorter=sorter)]
        print(f"i     index   rpeak")
        for i in range(len(idxs)):
            print(f"{i} {idxs[i]:>9} {ba[i]:>7}")
        # print(idxs)
    else:
        print('Tokių reikšmių nėra')





OS in my system :  linux
Skriptas rpeaks sulyginimui. Lyginamas rpeaks masyvas, suformuotas iš json failo,
nuskaityto iš Zive frontendo ir rpeaks masyvas, gautas iš EKG signalo naudojant
Neurokit2 funkciją ecg_peaks. EKG signalas (įrašas) irgi gautas iš Zive frontendo.
Abu rpeaks masyvai įrašomi į json vizualiniam sulyginimui su http://www.jsondiff.com/
rpeaks iš json failo varde pridėta '-js', rpeaks iš signalo - pridėta '-sg'

Zive įrašas:  1637624.073
rpeaks iš json failo: 830
rpeaks iš signal: 667

Reikšmės faile rpeaks_from_json kurių nėra faile rpeaks_from_signal
i     index   rpeak
0         0      82
1         1     277
2         2     460
3         3     650
4         4     840
5         5    1029
6         6    1219
7         7    1420
8         8    1626
9         9    1835
10        10    2034
11        11    2239
12        12    2447
13        13    2653
14        14    2849
15        15    3050
16        16    3249
17        17    3440
18        18    3638
19        19   

In [2]:
for file_name in list:
    file_name = str(file_name)    
    print(f"\nZive įrašas:  {file_name:>2}")

        # I-a dalis: nuskaitome rpeaks iš json failo

    filepath = Path(rec_dir, file_name + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_dict = data['rpeaks']
    rpeaks_from_json = np.array([dict['sampleIndex'] for dict in rpeaks_dict])
    # print(rpeaks_from_json[:5])

            # II-a dalis: suformuojame rpeaks su Neurokitu

    filepath = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(filepath)
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    # print(rpeaks_from_signal[:5])
    
  
            # Sulyginimas

    ab = np.setdiff1d(rpeaks_from_json, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,rpeaks_from_json)
    print(f"Nesutampančių rpeaks skaičius NS: {np.size(ab)+np.size(ba)} ")





Zive įrašas:  1637624.073
Nesutampančių rpeaks skaičius NS: 1481 
